In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [2]:
train = pd.read_csv(r'D:\바탕화면\지후의 생각\9-12공부\dacon\train.csv')
test = pd.read_csv(r'D:\바탕화면\지후의 생각\9-12공부\dacon\test.csv')
print(train.shape, test.shape)

(20000, 23) (10000, 22)


In [8]:
X_train = train.drop('Income',axis=1)
y_train = train['Income']

In [9]:
X_train.head(5)

,ID,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,...,Household_Summary,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Gains,Losses,Dividends,Income_Status
0,TRAIN_00000,63,M,Middle (7-8),Full-Time,4,Social Services,Services,White,All other,...,Householder,Native,US,US,US,Nonfiler,0,0,0,Unknown
1,TRAIN_00001,37,M,Associates degree (Vocational),Full-Time,52,Entertainment,Services,White,All other,...,Householder,Native,US,US,US,Single,0,0,0,Under Median
2,TRAIN_00002,58,F,High graduate,Full-Time,52,Manufacturing (Non-durable),Admin Support (include Clerical),Black,All other,...,Householder,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),3411,0,0,Under Median
3,TRAIN_00003,44,M,High graduate,Full-Time,52,Retail,Technicians & Support,White,All other,...,Householder,Native,US,US,US,Single,0,0,0,Under Median
4,TRAIN_00004,37,F,High graduate,Full-Time,52,Retail,Sales,White,All other,...,Householder,Native,US,US,US,Head of Household (HOH),0,0,0,Unknown


In [4]:
def age(x):
    if x <= 20:
        return 0
    elif 20 < x <= 30:
        return 1
    elif 30 < x <= 50:
        return 2
    else:
        return 3

def education_group(education):
    if 'Children' in education or 'Middle' in education or 'Elementary' in education or 'Kindergarten' in education :
        return 0
    elif 'High graduate' in education:
        return 1
    elif 'degree' in education or 'High' in education: 
        return 2
    else:
        return 4
    
def employment_group(employment):
    if 'Children' in employment  :
        return 0
    elif 'Seeking' in employment or 'Not Working' in employment:
        return 1
    else:
        return 2
    
def working_week(hour):
    if hour == 0  :
        return 0
    elif 0 < hour <52 :
        return 1
    else:
        return 2

In [11]:
X_train.head(3)

,ID,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,...,Household_Summary,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Gains,Losses,Dividends,Income_Status
0,TRAIN_00000,63,M,Middle (7-8),Full-Time,4,Social Services,Services,White,All other,...,Householder,Native,US,US,US,Nonfiler,0,0,0,Unknown
1,TRAIN_00001,37,M,Associates degree (Vocational),Full-Time,52,Entertainment,Services,White,All other,...,Householder,Native,US,US,US,Single,0,0,0,Under Median
2,TRAIN_00002,58,F,High graduate,Full-Time,52,Manufacturing (Non-durable),Admin Support (include Clerical),Black,All other,...,Householder,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),3411,0,0,Under Median


In [5]:
label_encoder = LabelEncoder()
X_train['Gender'] = label_encoder.fit_transform(X_train['Gender'])
X_train['Race'] = label_encoder.fit_transform(X_train['Race'])
X_train['Age'] = X_train['Age'].apply(age)
X_train['Education_Status'] = X_train['Education_Status'].apply(education_group)
X_train['Employment_Status'] = X_train['Employment_Status'].apply(employment_group)
X_train['Working_Week (Yearly)'] = X_train['Working_Week (Yearly)'].apply(working_week)
X_train = X_train.iloc[:,1:6]
X_train

,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly)
0,3,1,0,2,1
1,2,1,2,2,2
2,3,0,1,2,2
3,2,1,1,2,2
4,2,0,1,2,2
...,...,...,...,...,...
19995,2,1,1,0,2
19996,0,0,4,2,1
19997,1,1,4,0,2
19998,3,0,1,1,0


In [222]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [224]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape,X_val.shape,y_val.shape)

(12800, 5) (3200, 5) (12800,) (3200,) (4000, 5) (4000,)


In [212]:
model = XGBRegressor(iterations=3000, 
                          learning_rate=0.01, 
                          random_seed=416)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=100)

[0]	validation_0-rmse:700.17617


[99]	validation_0-rmse:631.02305


d:\program\Anaconda\envs\home\lib\site-packages\xgboost\core.py:160: UserWarning: [23:26:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "iterations", "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, iterations=3000, learning_rate=0.01,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, ...)

In [242]:
model = CatBoostRegressor(iterations=1000, 
                          learning_rate=0.1, 
                          random_seed=416)
model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=100)

0:	learn: 694.4504625	test: 688.1258492	best: 688.1258492 (0)	total: 2.38ms	remaining: 2.38s
100:	learn: 623.1910376	test: 616.5417875	best: 616.4885529 (81)	total: 154ms	remaining: 1.38s
200:	learn: 622.2966100	test: 617.2735542	best: 616.4885529 (81)	total: 322ms	remaining: 1.28s
300:	learn: 622.0735981	test: 617.6321508	best: 616.4885529 (81)	total: 487ms	remaining: 1.13s
400:	learn: 621.9539724	test: 617.8078160	best: 616.4885529 (81)	total: 666ms	remaining: 995ms
500:	learn: 621.8900246	test: 617.9322397	best: 616.4885529 (81)	total: 914ms	remaining: 910ms
600:	learn: 621.8485128	test: 618.0091027	best: 616.4885529 (81)	total: 1.1s	remaining: 729ms
700:	learn: 621.8168354	test: 618.0598540	best: 616.4885529 (81)	total: 1.33s	remaining: 567ms
800:	learn: 621.7933310	test: 618.1036392	best: 616.4885529 (81)	total: 1.53s	remaining: 381ms
900:	learn: 621.7758784	test: 618.1311799	best: 616.4885529 (81)	total: 1.7s	remaining: 187ms
999:	learn: 621.7657675	test: 618.1484420	best: 616.48

In [245]:
from sklearn.model_selection import GridSearchCV

# CatBoostRegressor 모델 객체 생성
model = CatBoostRegressor()

# 탐색할 하이퍼파라미터 그리드 설정
param_grid = {
    'iterations': [500, 1000,3000, 10000],  # 반복 횟수
    'learning_rate': [0.01, 0.05, 0.1],  # 학습률
    'depth': [4, 6, 8],  # 트리의 최대 깊이
    'l2_leaf_reg': [0.1, 1, 10],  # L2 정규화 파라미터
    'bagging_temperature': [0.1, 0.5, 1]  # 베깅 온도
}

# GridSearchCV를 사용하여 최적의 하이퍼파라미터 탐색
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Parameters:", grid_search.best_params_)

# 최적의 모델 추출
best_model = CatBoostRegressor(**grid_search.best_params_)
best_model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=100)

# 최적의 모델로 예측 수행
y_pred = best_model.predict(X_test)


0:	learn: 706.2878616	total: 1.55ms	remaining: 1.55s
1:	learn: 705.0837352	total: 3.71ms	remaining: 1.85s
2:	learn: 703.8484418	total: 5.19ms	remaining: 1.72s
3:	learn: 702.6229381	total: 6.37ms	remaining: 1.59s
4:	learn: 701.4567986	total: 7.61ms	remaining: 1.51s
5:	learn: 700.2396984	total: 9.01ms	remaining: 1.49s
6:	learn: 699.0529775	total: 10.3ms	remaining: 1.46s
7:	learn: 697.8881932	total: 11.6ms	remaining: 1.43s
8:	learn: 696.7510457	total: 12.8ms	remaining: 1.41s
9:	learn: 695.7047691	total: 13.9ms	remaining: 1.38s
10:	learn: 694.5982626	total: 15.1ms	remaining: 1.36s
11:	learn: 693.5134794	total: 16.6ms	remaining: 1.37s
12:	learn: 692.4508042	total: 17.9ms	remaining: 1.36s
13:	learn: 691.4008188	total: 19.1ms	remaining: 1.35s
14:	learn: 690.3889695	total: 20.3ms	remaining: 1.33s
15:	learn: 689.3637828	total: 21.6ms	remaining: 1.33s
16:	learn: 688.3957925	total: 23.2ms	remaining: 1.34s
17:	learn: 687.4175500	total: 24.6ms	remaining: 1.34s
18:	learn: 686.4651758	total: 25.8ms	r

In [251]:
grid_search.best_params_

{'bagging_temperature': 0.1,
 'depth': 4,
 'iterations': 1000,
 'l2_leaf_reg': 10,
 'learning_rate': 0.01}

{'bagging_temperature': 0.1,
 'depth': 4,
 'iterations': 1000,
 'l2_leaf_reg': 10,
 'learning_rate': 0.01}

In [253]:
y_pred = best_model.predict(X_test)

In [254]:
pd.concat([pd.DataFrame(y_pred),pd.DataFrame(y_test.values)],axis=1)

,0,0
0,32.919522,0
1,1.002935,0
2,734.218798,525
3,621.442244,1000
4,493.541460,700
...,...,...
3195,830.842375,700
3196,690.972874,772
3197,50.280323,0
3198,1.002935,0


In [255]:
test = pd.read_csv(r'D:\바탕화면\지후의 생각\9-12공부\dacon\test.csv')

label_encoder = LabelEncoder()
test['Gender'] = label_encoder.fit_transform(test['Gender'])
test['Race'] = label_encoder.fit_transform(test['Race'])
test['Age'] = test['Age'].apply(age)
test['Education_Status'] = test['Education_Status'].apply(education_group)
test['Employment_Status'] = test['Employment_Status'].apply(employment_group)
test['Working_Week (Yearly)'] = test['Working_Week (Yearly)'].apply(working_week)
test = test.iloc[:,1:6]
test

,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly)
0,3,1,2,0,0
1,2,1,0,0,0
2,0,0,2,0,2
3,2,0,2,2,1
4,0,1,0,0,0
...,...,...,...,...,...
9995,2,1,2,0,2
9996,1,1,4,0,2
9997,0,1,1,0,1
9998,0,1,0,0,0


In [256]:
y_pred = best_model.predict(test)

In [257]:
sub = pd.read_csv(r'D:\바탕화면\지후의 생각\9-12공부\dacon\sample_submission.csv')
sub['Income']=y_pred
sub.to_csv('jihoo3.csv',index = False)